In [141]:
# TO DO:
# Sorter per kandidat
# Inkluder timestamp for hver post




In [3]:
import numpy as np
import json
import os
import time
import string
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import re
import pickle
from operator import itemgetter

# Alpha-num, lowercase
def easy_clean(comment):
    comment = re.sub('[^A-Za-z ]+', '', comment['body']).lower().split(' ')
    #comment = [re.sub('[^A-Za-z]+', '', word) for word in comment]
    return comment

# nltk tokenize, stopwords
def nltk_clean(comment):
    #comment = comment
    #comment = nltk.sent_tokenize(comment)
    print(comment)
    comment = nltk.tokenize.word_tokenize(comment)
    comment = [word for word in comment if word.isalpha()]
    stop_words = nltk.corpus.stopwords.words('english')
    comment = [word for word in comment if word not in stop_words]
    return comment

# Remove quotes, clean text using translation table (punctuations)
def nltk2_clean(comment): #tokenize, removes case, remove special characters and numbers
    comment = [e for e in comment.splitlines() if e[:2] != '&g']
    comment = ' '.join(comment)
    comment = comment.split(' ')
    comment = [w.lower() for w in comment]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in comment]
    words = [word for word in stripped if word.isalpha()]
    return ' '.join(words)

# Tokenize at sentence level, applies nltk2_clean on each sentence
def clean_sent(comment): #tokenize, removes case, remove special characters and numbers
    comment = comment['body']
    comments = nltk.sent_tokenize(comment)
    comments = [nltk2_clean(sent) for sent in comments]
    return comments


# SÆTNINGSDELING REGEX: (?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s

#    sentence_list = tokenize.sent_tokenize(paragraph)
#    paragraphSentiments = 0.0
#    for sentence in sentence_list:
#        vs = analyzer.polarity_scores(sentence)
#        print("{:-<69} {}".format(sentence, str(vs["compound"])))
#        paragraphSentiments += vs["compound"]
#    print("AVERAGE SENTIMENT FOR PARAGRAPH: \t" + str(round(paragraphSentiments / len(sentence_list), 4)))





In [3]:
# Remove duplicates across dataSet
clSet = [] 
crSet = []
trSet = []
saSet = []

print('starter')
dataSet = []
start = time.time()
n = 0
for file in os.listdir('data/'):
    with open('data/'+file) as f:
        data = json.load(f)
    
    if file[:2] == 'cl':
        candidates =  ['bernie' ,'sanders' ,'donald' ,'trump' ,'ted' ,'cruz']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        clData = [[e,e['created_utc']] for e in data if not any(item in candidates for item in easy_clean(e))]
        clSet = clSet + clData
        
    elif file[:2] == 'cr':
        candidates =  ['bernie' ,'sanders' ,'donald' ,'trump', 'hillary', 'clinton']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        crData = [[e,e['created_utc']] for e in data if not any(item in candidates for item in easy_clean(e))]
        crSet = crSet + crData

    elif file[:2] == 'tr':
        candidates =  ['bernie' ,'sanders' ,'ted' ,'cruz', 'hillary', 'clinton']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        trData = [[e,e['created_utc']] for e in data if not any(item in candidates for item in easy_clean(e))]
        trSet = trSet + trData
        
    elif file[:2] == 'sa':
        candidates =  ['hillary', 'clinton' ,'donald' ,'trump' ,'ted' ,'cruz']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        saData = [[e,e['created_utc']] for e in data if not any(item in candidates for item in easy_clean(e))]
        saSet = saSet+saData
    print(file + '\ndone\n')
    
end = time.time()
print('...Done in '+str(end - start))



starter
clinton_politics_2016-01-01_2016-02-01.json
done

clinton_politics_2016-02-01_2016-03-01.json
done

clinton_politics_2016-03-01_2016-04-01.json
done

clinton_politics_2016-04-01_2016-05-01.json
done

clinton_politics_2016-05-01_2016-06-01.json
done

clinton_politics_2016-06-01_2016-07-01.json
done

clinton_politics_2016-07-01_2016-08-01.json
done

clinton_politics_2016-08-01_2016-09-01.json
done

clinton_politics_2016-09-01_2016-10-01.json
done



KeyboardInterrupt: 

In [5]:
# Remove duplicates across dataSet
clSet = [] 
crSet = []
trSet = []
saSet = []

print('starter')
dataSet = []
start = time.time()
n = 0
for file in os.listdir('data/'):
    with open('data/'+file) as f:
        data = json.load(f)
    
    if file[:2] == 'cl':
        candidates =  ['bernie' ,'sanders' ,'donald' ,'trump' ,'ted' ,'cruz']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        clData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        clSet = clSet + clData
        
    elif file[:2] == 'cr':
        candidates =  ['bernie' ,'sanders' ,'donald' ,'trump', 'hillary', 'clinton']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        crData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        crSet = crSet + crData

    elif file[:2] == 'tr':
        candidates =  ['bernie' ,'sanders' ,'ted' ,'cruz', 'hillary', 'clinton']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        trData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        trSet = trSet + trData
        
    elif file[:2] == 'sa':
        candidates =  ['hillary', 'clinton' ,'donald' ,'trump' ,'ted' ,'cruz']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        saData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        saSet = saSet+saData
    print(file + '\ndone\n')
    
end = time.time()
print('...Done in '+str(end - start))



starter
clinton_politics_2016-01-01_2016-02-01.json
done

clinton_politics_2016-02-01_2016-03-01.json
done

clinton_politics_2016-03-01_2016-04-01.json
done

clinton_politics_2016-04-01_2016-05-01.json
done

clinton_politics_2016-05-01_2016-06-01.json
done

clinton_politics_2016-06-01_2016-07-01.json
done

clinton_politics_2016-07-01_2016-08-01.json
done

clinton_politics_2016-08-01_2016-09-01.json
done

clinton_politics_2016-09-01_2016-10-01.json
done

clinton_politics_2016-10-01_2016-11-01.json
done

clinton_politics_2016-11-01_2016-12-01.json
done

clinton_politics_2016-12-01_2016-12-31.json
done

cruz_politics_2016-01-01_2016-02-01.json
done

cruz_politics_2016-02-01_2016-03-01.json
done

cruz_politics_2016-03-01_2016-04-01.json
done

cruz_politics_2016-04-01_2016-05-01.json
done

cruz_politics_2016-05-01_2016-06-01.json
done

cruz_politics_2016-06-01_2016-07-01.json
done

cruz_politics_2016-07-01_2016-08-01.json
done

cruz_politics_2016-08-01_2016-09-01.json
done

cruz_politics_20

In [6]:
# Clean data
dataSet = [clSet, crSet, trSet, saSet]
dataSet = [[[clean_sent(comment),comment['created_utc']] for comment in candidate] for candidate in dataSet]


In [14]:
print(dataSet[0][0<<][0])

['you are twisting them by equating regular staffers versus astroturfing', 'so are you saying its ethical for a presidential candidate to try and manipulate public perception through anonymous boards', 'it doesnt matter if astrotufing gains hillary vote or votes', 'its not ethical to employ individuals to act as regular people with no direct affiliation']


In [11]:
# Save data
with open("nodupData", "wb") as fp:
    pickle.dump(dataSet, fp)

In [6]:
with open(r"nodupData", "rb") as input_file:
    dataSet = pickle.load(input_file)
    

In [24]:

analyzer = SIA()
#sent = [[' '.join(sentence), analyzer.polarity_scores(sentence[0])] for sentence in cleanData]

# Calculate polarity score of each sentence
#sentScores = [[[[[sentence, analyzer.polarity_scores(sentence)['compound']],comment[1]]
#             for sentence in comment if len(sentence)>1] for comment in cleanData] for cleanData in dataSet]

sentScores = [[[[[sentence, analyzer.polarity_scores(sentence)['compound']] 
                 for sentence in comment[0] if len(sentence)>1],comment[1]]
               for comment in candidate]
              for candidate in dataSet]


In [31]:
print(sentScores[0][0][0])

[['you are twisting them by equating regular staffers versus astroturfing', 0.0], ['so are you saying its ethical for a presidential candidate to try and manipulate public perception through anonymous boards', 0.5106], ['it doesnt matter if astrotufing gains hillary vote or votes', 0.3239], ['its not ethical to employ individuals to act as regular people with no direct affiliation', -0.5996]]


In [46]:

# Calculate average polarity score for each sentence in comment + comment in string format
commentScores = []
for candidate in sentScores:
    candidateScores = []
    for comment in candidate:
        sentence = '. '.join([sentence[0] for sentence in comment[0]])
        avg = np.mean([sentence[1] for sentence in comment[0]])
        stamp = comment[1]
        candidateScores.append([sentence, avg, stamp])
    commentScores.append(candidateScores)






In [84]:
# Save data
with open("commentScores_all", "wb") as fp:
    pickle.dump(commentScores, fp)

In [83]:
#print(commentScores[0][0])

avg = [[score[1] for score in candidate] for candidate in commentScores]


test = [e for e in avg[0] if type(e)!= np.float64]
print(True in np.isnan(avg[0]))
np.argwhere(np.isnan(avg[0]))

print(np.nanmean(avg[0]))
print(np.nanmean(avg[1]))
print(np.nanmean(avg[2]))
print(np.nanmean(avg[3]))

print(commentScores[2][0])

True
0.0128060096196
0.0101767257808
0.00561841407161
0.0657188424828
['see. im different. i vote for the person that i think will help the country the most. not who helps me personally the most. im not selfish and that why im voting for donald trump. i want his leadership for everybody', 0.09318333333333334, 1452759802]


In [ ]:
# Sort by compound

commentScores.sort(key=lambda e: e[1],reverse=True)

i = -9
print(commentScores[i])

In [136]:
import pandas as pd

test = pd.DataFrame(commentScores[2])
test2 = test[(test[2] < 1454280854) & (test[2] > 1454279996)]
test2 = test[test[1] > 0.98]

test2

,0,1,2
78905,look youre entitled to your opinion thats fine...,0.9966,1456036722
83531,just this month he raised million for the foll...,0.9933,1456518127
96515,he also said he was joking when he said that i...,0.9817,1458603093
106919,the best thing fox could do would be to accept...,0.9814,1457159489
117457,bravely bold sir donald rode forth from trumpa...,0.9927,1457817614
170638,bravely brave sir donald ran away bravely bold...,0.9936,1457815676
197840,trump immediately comes out meandering but hes...,0.9830,1457057182
201529,aint trumptards repeating cock cock cock cock ...,0.9965,1461905141
203340,trump is a cartoon outsider who is conservativ...,0.9811,1460003650
247950,steven m colloton us court of appeals judge ei...,0.9882,1463597446
